<a href="https://colab.research.google.com/github/CalculatedContent/ww-phys_theory/blob/master/Jacobian_Paper_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook to Test the Jacobian Theory Generalization Paper

We take the expected value of the Jacobian over the labels

$\Vert\mathbf{J}^{T}\mathbf{y}\Vert$

I'm guessing we should use the predicted labels 







In [16]:
Jdir =  '/content/drive/My Drive/J_paper_test'

thismodel = 'resnet20_cifar10' #'resnet20_cifar10','resnet56_cifar10','resnet110_cifar10','resnet164bn_cifar10', 'resnet272bn_cifar10']
num_classes = 10

num_data = 50000
batch_size = 100

device = 'cuda:0'


time: 2.13 ms


In [17]:
!pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
time: 2.54 s


In [18]:
!pip install ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.61 s


In [19]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 1.59 ms


In [20]:
!ls "$Jdir"

time: 854 ms


In [21]:
!pip install pytorchcv
!pip install powerlaw
!pip install GPUtil
import GPUtil

time: 7.05 s


In [22]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

import math
import copy

import gc


time: 4.55 ms


### Run InstallPhysTheory notebook 

In [23]:
import sys
sys.path.append("/content/drive/My Drive/Jacobian")
import jacobian as jac
import utils

time: 1.04 ms


In [24]:
def construct_paperJ( model, data_loader, batch_size, device='cuda:0', num_classes=num_classes, data_dim = 3*32*32):
  """Constructs J matrix for the paper 
  Input: Model, data_loader, batch_size, device, num_classes, data_dim.
  Optional Arguments: device, num_classes (default 10), data_dim (default: 3072).
  Return: expected value  (len(data_loader)*num_classes*batch_size, num_classes*data_dim).
  """
  Js = []
  model.eval()
  model = model.to(device)

  for batch, data in enumerate(data_loader):
    features, labels = data
    inputs = features.to(device)
    inputs.requires_grad=True
    outputs = model(inputs)
    J = jac.batch_diagJ(inputs, outputs)
    Js.append(J)

  full_J = torch.stack(Js, dim=0)
  full_J = full_J.reshape(len(data_loader)*batch_size*num_classes, data_dim)

  return full_J

time: 8.7 ms


In [25]:
def jacobian_rect(model, data_loader, batch_size, num_classes=num_classes, device='cuda:0'): 
  '''compute a full of J Rectangular Jacobian directly.  '''

  model.eval()
  model = model.to(device)

  J = construct_paperJ(model, data_loader, batch_size=batch_size, device=device) 
 
  gc.collect()
  torch.cuda.empty_cache()

  return J.detach().cpu().numpy()

time: 3.43 ms


### ResNet 

In [26]:
import pytorchcv
from pytorchcv.model_provider import get_model as ptcv_get_model

time: 843 µs


### Compute JJ, in batches or all at once


In [27]:
def get_datasets(thismodel, batch_size):
  print("getting dataset for ", thismodel)
  if thismodel.endswith('cifar100'):
    print("loading cifar100")
    train_dataset = datasets.CIFAR100(
      root='data', 
      train=True, 
      transform=transforms.ToTensor(),
      download=True)
    test_dataset = datasets.CIFAR100(
      root='data', 
      train=False, 
      transform=transforms.ToTensor(),
      download=True) 
 
    train_loader = DataLoader(
      dataset=train_dataset, 
      batch_size=batch_size,
      num_workers=4,
      shuffle=False)
    test_loader = DataLoader(
      dataset=test_dataset, 
      batch_size=batch_size,
      num_workers=4,
      shuffle=False)

  else:
    print("loading cifar10")
    train_dataset, test_dataset, train_loader, test_loader = utils.get_data(batch_size=batch_size)

      
  return train_dataset, test_dataset, train_loader, test_loader

time: 9.09 ms


In [ ]:
print(thismodel)
train_dataset, test_dataset, train_loader, test_loader = get_datasets(thismodel, batch_size)#utils.get_data(batch_size=batch_size)
#train_dataset, test_dataset, train_loader, test_loader = utils.get_data(batch_size=batch_size)

print(test_dataset)
GPUtil.showUtilization()


resnet20_cifar10
getting dataset for  resnet20_cifar10
loading cifar10


In [ ]:
model = ptcv_get_model(thismodel, pretrained=True)  


In [ ]:
J_full = construct_paperJ(model, train_loader, batch_size)

In [ ]:
J_full.shape

## Computing the expected values might be easier to do just do in batch

In [ ]:
import sklearn
from sklearn.decomposition import TruncatedSVD

In [ ]:
J = J_full.detach().cpu().numpy()
J.shape

In [ ]:
del J_full
gc.collect()
torch.cuda.empty_cache()

In [ ]:
JJ = np.dot(np.transpose(J), J)

In [ ]:
JJ.shape

In [ ]:
svals = np.linalg.svd(JJ,  compute_uv=False)

In [ ]:
svals = svals / J.shape[0]

In [ ]:
plt.hist(svals,bins=100);
plt.title("JJ eigenvales, J (kn x d) \n "+thismodel)

In [ ]:
plt.hist(np.log10(svals),bins=100);
plt.title("log10 JJ eigenvales, J (kn x d) \n "+thismodel)


In [ ]:
import powerlaw
fit = powerlaw.Fit(svals)
print(fit.alpha)
print(fit.xmin)
R, p = fit.distribution_compare('truncated_power_law', 'lognormal')
print(R,p)

In [ ]:
fig2 = fit.plot_pdf(color='b', linewidth=2)
fit.power_law.plot_pdf(color='b', linestyle='--', ax=fig2)
fit.plot_ccdf(color='r', linewidth=2, ax=fig2)
fit.power_law.plot_ccdf(color='r', linestyle='--', ax=fig2)
plt.title(thismodel)
plt.show()
